In [1]:
!pip install uv
!uv sync

/bin/bash: line 1: pip: command not found
Resolved 145 packages in 1ms
Uninstalled 20 packages in 29ms
 - asttokens==3.0.1
 - comm==0.2.3
 - debugpy==1.8.17
 - executing==2.2.1
 - ipykernel==7.1.0
 - ipython==9.7.0
 - ipython-pygments-lexers==1.1.1
 - jedi==0.19.2
 - jupyter-client==8.6.3
 - jupyter-core==5.9.1
 - nest-asyncio==1.6.0
 - parso==0.8.5
 - pexpect==4.9.0
 - prompt-toolkit==3.0.52
 - ptyprocess==0.7.0
 - pure-eval==0.2.3
 - pyzmq==27.1.0
 - stack-data==0.6.3
 - tornado==6.5.2
 - wcwidth==0.2.14


In [2]:
#download our dataset and all models.
# !uv run scripts/download_gdrive.py 

In [9]:
from scripts.download_gdrive import download_dataset, download_models
RTFS_MODEL_PATH="data/models/rtfs-4-reuse.pth" #Our SOTA Model
DPRNN_MODEL_PATH="data/models/dprnn.pth" #Our best dprnn checkpoint
DATASET_PATH="data/datasets/dla_dataset_small_av"
GDRIVE_URLS = {
    "models": {
        # rtfs-4-reuse
        "https://drive.google.com/uc?id=1lzO_cyN0WKof1rtnCPZpN-BHcELO-85c": RTFS_MODEL_PATH,
        "https://drive.google.com/uc?id=1IU4mFseGPtac6g58jTS48GzlMI1wMJQ8":DPRNN_MODEL_PATH
    },
    "dataset": {
        "https://drive.google.com/uc?id=178uAdx9S6lln1z0Yr9ws-tBpmDkSuHqD":DATASET_PATH
    }
}
download_models(GDRIVE_URLS)
# download_dataset(GDRIVE_URLS)

Downloading...
From (original): https://drive.google.com/uc?id=1lzO_cyN0WKof1rtnCPZpN-BHcELO-85c
From (redirected): https://drive.google.com/uc?id=1lzO_cyN0WKof1rtnCPZpN-BHcELO-85c&confirm=t&uuid=163ff056-649b-4345-bdae-261d11db8f56
To: /home/torfinhell/Ml/DSP/AVSS/data/models/rtfs-4-reuse.pth
100%|██████████| 71.2M/71.2M [00:36<00:00, 1.97MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1IU4mFseGPtac6g58jTS48GzlMI1wMJQ8
From (redirected): https://drive.google.com/uc?id=1IU4mFseGPtac6g58jTS48GzlMI1wMJQ8&confirm=t&uuid=fd5fe1a8-0a48-428f-8ae2-aade4927ad93
To: /home/torfinhell/Ml/DSP/AVSS/data/models/dprnn.pth
100%|██████████| 31.7M/31.7M [00:19<00:00, 1.61MB/s]


In [10]:
#you can specify a link to your yandex disk zip dataset for it to download during training ot infernce
!export YANDEX_DISK_URL=https://disk.360.yandex.ru/d/5pz96ysIZi33IQ

In [13]:
!uv run inference.py inferencer.save_path="pred_dprnn_small_av"  inferencer.from_pretrained={DPRNN_MODEL_PATH} download_name=dla_dataset_small_av model=dprnn device="cpu" -cn=inference

Could not override 'device'.
To append to your config use +device=cpu
Key 'device' is not in struct
    full_key: device
    object_type=dict

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.


In [14]:
!uv run inference.py inferencer.save_path="pred_rtfs_small_av"  inferencer.from_pretrained={RTFS_MODEL_PATH} download_name=dla_dataset_small_av model=rtfs-4-reuse -cn=inference

Using GPU: cuda:0 with 7.51 GB free
Creating dataset: 100%|██████████████████| 3000/3000 [00:00<00:00, 12054.96it/s]
RTFSModel(
  (stft): TransformSTFT()
  (audio_encoder): RTFSAudioEncoder(
    (conv): Conv2d(2, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (ap_block): RTFSBlock(
    (compressor): CompressorBlock(
      (dimension_compression_layer): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
      (tf_compression_layers): Sequential(
        (0): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), groups=128)
        (1): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), groups=128)
      )
    )
    (reconstructor): ReconstructionBlock(
      (g_fuse_layers): ModuleList(
        (0-2): 3 x TFARUnit(
          (w1_pathway): Sequential(
            (0): Conv2d(128, 128, kernel_size=(4, 4), stride=(1, 1), padding=same, groups=128)
            (1): GlobalLayerNorm(1, 128, eps=1e-08, affine=True)
            (2): Sigmoid()
      

In [24]:
from src.utils.io_utils import ROOT_PATH
#calculate metrics on predictions on the dla_dataset_small_av folder of rtfs model 
!uv run calc_metrics.py --gt_s1=data/datasets/dla_dataset_small_av/audio/val/s1 --gt_s2=data/datasets/dla_dataset_small_av/audio/val/s2 \
    --pred_s1=data/saved/pred_small_av/val/s1 --pred_s2=data/saved/pred_small_av/val/s1 

Calculating Metrics...: 100%|█████████████████| 500/500 [01:53<00:00,  4.41it/s]
Average Metrics:
STOI: 0.5363
PESQ: 1.5642
SI-SDR-I: -10.0283
SI-SNR-I: -10.0285


In [ ]:
import time
start = time.time()

!uv run inference.py inferencer.save_path="pred_rtfs_small_av" \
    inferencer.from_pretrained={RTFS_MODEL_PATH} \
    download_name=dla_dataset_small_av dataloader=inference_dataloaders\
    model=rtfs-4-reuse inferencer.device=cpu  dataloader.val_dataloader.batch_size=8 \
     dataloader.test_dataloader.batch_size=8 +datasets.val.limit=8 +datasets.test.limit=8 -cn=inference

end = time.time()
print("Elapsed:", end - start, "seconds")

Creating dataset: 100%|██████████████████| 3000/3000 [00:00<00:00, 12023.52it/s]
RTFSModel(
  (stft): TransformSTFT()
  (audio_encoder): RTFSAudioEncoder(
    (conv): Conv2d(2, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (ap_block): RTFSBlock(
    (compressor): CompressorBlock(
      (dimension_compression_layer): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
      (tf_compression_layers): Sequential(
        (0): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), groups=128)
        (1): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), groups=128)
      )
    )
    (reconstructor): ReconstructionBlock(
      (g_fuse_layers): ModuleList(
        (0-2): 3 x TFARUnit(
          (w1_pathway): Sequential(
            (0): Conv2d(128, 128, kernel_size=(4, 4), stride=(1, 1), padding=same, groups=128)
            (1): GlobalLayerNorm(1, 128, eps=1e-08, affine=True)
            (2): Sigmoid()
          )
          (w2_pathway): Sequen